# Advection and ML heating/cooling

# binned stats of ML heat balance

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cmocean as cmo
import glob
from tqdm import tqdm
import sys
import calendar

In [18]:
case_name = 'Strong'
years = '[2-3]'

In [19]:
x_array = np.arange(0, 1122*2500, 2500)
y_array = np.arange(20*2500, 430*2500, 2500)
x_rho, y_rho = np.meshgrid(x_array, y_array)

# Monthly mean temp_rate and eddy advection

In [20]:
heat_balance_files = glob.glob('../monthly_ml_heat_budget_data_small_mesoscale/' 
                               + case_name + '/monthly_ml_heat_balance_2016-*_' + case_name + 'Eddies.nc')
heat_balance_files = sorted(heat_balance_files)
heat_balance_files

['../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-01_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-02_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-03_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-04_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-05_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-06_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-07_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-08_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Strong/monthly_ml_heat_balance_2016-09_StrongEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesosca

In [21]:
%%time
temp_rate_ml, eddy_adv_ml, temp_adv_ml = np.zeros([len(heat_balance_files), 410, 1122]), np.zeros([len(heat_balance_files), 410, 1122]), np.zeros([len(heat_balance_files), 410, 1122])
for num, temp_rate_ml_eddy_file in enumerate(heat_balance_files):
    #print(temp_rate_ml_eddy_file)
    temp_rate_ml_ds = xr.open_dataset(temp_rate_ml_eddy_file)
    temp_rate_ml[num] = temp_rate_ml_ds.temp1_rate_ml.data
    eddy_adv_ml[num] = temp_rate_ml_ds.eddy_adv_ml.data
    #temp_adv_ml[num] = temp_rate_ml_ds.temp_adv_ml.data

CPU times: user 212 ms, sys: 84.3 ms, total: 296 ms
Wall time: 1.38 s


In [22]:
MS_ds = xr.Dataset(data_vars={'eddy_adv_ml' : (('ocean_time','eta_rho', 'xi_rho'), eddy_adv_ml[:, 60:350, 60:1062]),
                            'temp_rate_ml' : (('ocean_time','eta_rho', 'xi_rho'), temp_rate_ml[:, 60:350, 60:1062]),
                            'x_rho' : (('eta_rho', 'xi_rho'), x_rho[60:350, 60:1062]),
                            'y_rho' : (('eta_rho', 'xi_rho'), y_rho[60:350, 60:1062])},
                coords={ 'eta_rho':   y_rho[60:350,0], 'xi_rho':    x_rho[0, 60:1062]})
MS_ds

<xarray.Dataset>
Dimensions:       (eta_rho: 290, ocean_time: 12, xi_rho: 1002)
Coordinates:
  * eta_rho       (eta_rho) int64 200000 202500 205000 ... 917500 920000 922500
  * xi_rho        (xi_rho) int64 150000 152500 155000 ... 2650000 2652500
Dimensions without coordinates: ocean_time
Data variables:
    eddy_adv_ml   (ocean_time, eta_rho, xi_rho) float64 -8.464e+03 ... -2.049...
    temp_rate_ml  (ocean_time, eta_rho, xi_rho) float64 25.3 27.67 ... -33.96
    x_rho         (eta_rho, xi_rho) int64 150000 152500 ... 2650000 2652500
    y_rho         (eta_rho, xi_rho) int64 200000 200000 200000 ... 922500 922500

## coefficients for binned statistics

In [23]:
population_cutoff = 5000
eddy_adv_bins = np.arange(-2000, 2000, 50)

# Eddy adv and ML heating/cooling

In [24]:
meso_grouped_mean = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).mean()
meso_grouped_std = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).std()

# calculating bin population
meso_groups = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).groups
    
count = 0
meso_bin_popu = np.zeros([len(meso_groups.keys()), 2])
for key in meso_groups.keys():
    meso_bin_popu[count] = key.left, len(meso_groups[key])
    count += 1
    
# finding the bins 
meso_valid = np.where(meso_bin_popu[meso_bin_popu[:, 0].argsort(), 1] > population_cutoff)

In [25]:
meso_grouped_mean

<xarray.Dataset>
Dimensions:           (eddy_adv_ml_bins: 79)
Coordinates:
  * eddy_adv_ml_bins  (eddy_adv_ml_bins) object (-2000, -1950] ... (1900, 1950]
Data variables:
    eddy_adv_ml       (eddy_adv_ml_bins) float64 -1.974e+03 ... 1.925e+03
    temp_rate_ml      (eddy_adv_ml_bins) float64 -110.6 -116.7 ... -6.017 -6.92
    x_rho             (eddy_adv_ml_bins) float64 1.511e+06 ... 1.912e+06
    y_rho             (eddy_adv_ml_bins) float64 8.447e+05 ... 8.902e+05

In [26]:
eddy_adv_ml_data, MS_SSTA_mean_data, MS_SSTA_std_data = meso_grouped_mean.eddy_adv_ml[meso_valid].data, meso_grouped_mean.temp_rate_ml[meso_valid].data, meso_grouped_std.temp_rate_ml[meso_valid].data
MS_bin_popu_data = meso_bin_popu[meso_bin_popu[:, 0].argsort(), 1][meso_valid]

In [27]:
eddy_stats_ds = xr.Dataset(data_vars={'eddy_adv_ml_bin_left' : (('temp_adv'), eddy_adv_ml_data),
                           'temp1_rate_ml_mean' : (('temp_adv'), MS_SSTA_mean_data),
                           'temp1_rate_ml_std' : (('temp_adv'), MS_SSTA_std_data),
                           'eddy_adv_ml_bin_population' : (('temp_adv'), MS_bin_popu_data)},
                coords={ 'eddy_adv': np.arange(len(MS_bin_popu_data))})
eddy_stats_ds

<xarray.Dataset>
Dimensions:                     (eddy_adv: 65, temp_adv: 65)
Coordinates:
  * eddy_adv                    (eddy_adv) int64 0 1 2 3 4 5 ... 60 61 62 63 64
Dimensions without coordinates: temp_adv
Data variables:
    eddy_adv_ml_bin_left        (temp_adv) float64 -1.675e+03 ... 1.524e+03
    temp1_rate_ml_mean          (temp_adv) float64 -74.7 -84.46 ... 10.5 5.614
    temp1_rate_ml_std           (temp_adv) float64 241.4 242.6 ... 177.4 179.7
    eddy_adv_ml_bin_population  (temp_adv) float64 5.282e+03 ... 5.054e+03

In [28]:
eddy_stats_ds.to_netcdf('./' + case_name + '_temp1_rate_eddy_adv_300km_binned_statistics_2016.nc')